In [5]:
import pandas as pd
import numpy as np
# from thangquang.bert_logistic import read_texts_from_dir
import os

def read_texts_from_dir(dir_path):
    """
    Reads the texts from a given directory and saves them in the pd.DataFrame with columns ['id', 'file_1', 'file_2'].

    Params:
      dir_path (str): path to the directory with data
    """
    # Count number of directories in the provided path
    dir_count = sum(
        os.path.isdir(os.path.join(root, d))
        for root, dirs, _ in os.walk(dir_path)
        for d in dirs
    )
    data = [0 for _ in range(dir_count)]
    print(f"Number of directories: {dir_count}")

    # For each directory, read both file_1.txt and file_2.txt and save results to the list
    i = 0
    for folder_name in sorted(os.listdir(dir_path)):
        folder_path = os.path.join(dir_path, folder_name)
        if os.path.isdir(folder_path):
            try:
                with open(
                    os.path.join(folder_path, "file_1.txt"), "r", encoding="utf-8"
                ) as f1:
                    text1 = f1.read().strip()
                with open(
                    os.path.join(folder_path, "file_2.txt"), "r", encoding="utf-8"
                ) as f2:
                    text2 = f2.read().strip()
                index = int(folder_name[-4:])
                data[i] = (index, text1, text2)
                i += 1
            except Exception as e:
                print(f"Error reading directory {folder_name}: {e}")

    # Change list with results into pandas DataFrame
    df = pd.DataFrame(data, columns=["id", "file_1", "file_2"]).set_index("id")
    return df

In [6]:
df_86 = pd.read_csv("/home/thangquang09/CODE/CTAI_MachineLearning/notebooks/submission_e5_catboost_improved.csv")
df_89 = pd.read_csv("/home/thangquang09/CODE/CTAI_MachineLearning/notebooks/submission_api_call_full_openai.csv")
df_91 = pd.read_csv("/home/thangquang09/CODE/CTAI_MachineLearning/notebooks/submission_api_call_full.csv")


In [7]:
df_86.rename(columns={'real_text_id': 'pred_86'}, inplace=True)
df_89.rename(columns={'real_text_id': 'pred_89'}, inplace=True)
df_91.rename(columns={'real_text_id': 'pred_91'}, inplace=True)

In [8]:
df_merged = pd.merge(df_86, df_89, on='id')
df_merged = pd.merge(df_merged, df_91, on='id')

print(df_merged.head())

   id  pred_86  pred_89  pred_91
0   0        2        2        2
1   1        2        2        2
2   2        1        1        1
3   3        1        1        1
4   4        2        2        2


In [14]:
y_pred1 = df_merged['pred_86'].values
y_pred2 = df_merged['pred_89'].values
y_pred3 = df_merged['pred_91'].values

In [15]:
import numpy as np
from sklearn.metrics import accuracy_score

# Giả sử bạn đã có dự đoán từ 3 mô hình dạng numpy arrays
# y_pred1, y_pred2, y_pred3 là mảng dự đoán của base models
# y_true là nhãn đúng

# Weighted voting – gán trọng số theo chính xác
weights = [86, 89, 91]  # Xem như tỉ lệ phần trăm thực tế
y_preds = np.array([y_pred1, y_pred2, y_pred3])

# Tính voting score cho từng dòng
n_samples = y_pred1.shape[0]
weighted_vote = np.zeros((n_samples,))

for i in range(n_samples):
    votes = {}
    for idx, pred in enumerate(y_preds[:, i]):
        votes[pred] = votes.get(pred, 0) + weights[idx]
    # Lấy nhãn có tổng weight cao nhất
    weighted_vote[i] = max(votes, key=votes.get)


In [17]:
df_merged['real_text_id'] = weighted_vote.astype(int)

In [20]:
df_merged[['id', 'real_text_id']].to_csv("simple_voting_868991.csv", index=False)

In [9]:
condition = (df_merged['pred_86'] == df_merged['pred_89']) & \
            (df_merged['pred_89'] == df_merged['pred_91'])

# Tạo consensus_df từ toàn bộ df_merged
consensus_df = df_merged.copy()

# Với những hàng thỏa mãn điều kiện, lấy giá trị từ pred_91
# Với những hàng không thỏa mãn, gán label là 3
consensus_df['ground_truth_guess'] = np.where(condition, consensus_df['pred_91'], 3)

print(f"Tổng số mẫu trong tập test: {len(df_merged)}")
print(f"Tìm thấy {condition.sum()} mẫu có sự đồng thuận cao.")
print(f"Có {(~condition).sum()} mẫu được gán label 3.")

# Đây là một phần của ground truth mà bạn có thể tin tưởng
print(consensus_df[['id', 'ground_truth_guess']].head())

Tổng số mẫu trong tập test: 1068
Tìm thấy 857 mẫu có sự đồng thuận cao.
Có 211 mẫu được gán label 3.
   id  ground_truth_guess
0   0                   2
1   1                   2
2   2                   1
3   3                   1
4   4                   2


In [13]:
consensus_df[['id', 'ground_truth_guess']].rename({"ground_truth_guess": "real_text_id"}).to_csv("X_test_ground_truth_with3.csv", index=False)

In [11]:
consensus_df

,id,pred_86,pred_89,pred_91,ground_truth_guess
0,0,2,2,2,2
1,1,2,2,2,2
2,2,1,1,1,1
3,3,1,1,1,1
4,4,2,2,2,2
...,...,...,...,...,...
1063,1063,1,1,1,1
1064,1064,1,1,1,1
1065,1065,1,1,1,1
1066,1066,2,2,2,2


In [12]:
df_test = read_texts_from_dir("/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/test")

Number of directories: 1068


In [15]:
# Lấy ra các index mà df_test trùng với consensus_df
new_df = pd.merge(consensus_df[['id', 'ground_truth_guess']], df_test, on='id')

In [19]:
new_df.to_csv("X_test_ground_truth.csv", index=False)